<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/cuda_topk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Oct 23 14:12:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://raw.githubusercontent.com/cteqeu/HAC/main/GPU/cuda_profile.cu

--2023-10-23 14:14:24--  https://raw.githubusercontent.com/cteqeu/HAC/main/GPU/cuda_profile.cu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2331 (2.3K) [text/plain]
Saving to: ‘cuda_profile.cu’

cuda_profile.cu     100%[===================>]   2.28K  --.-KB/s    in 0s      

2023-10-23 14:14:25 (38.8 MB/s) - ‘cuda_profile.cu’ saved [2331/2331]



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!nvcc cuda_profile.cu -o cuda_profile --generate-line-info

In [37]:
!compute-sanitizer --help

NVIDIA (R) Compute Sanitizer
Copyright (c) 2020-2022 NVIDIA Corporation
Version 2022.3.0
Usage: compute-sanitizer [options] [your-program] [your-program-options]

General options:
  -h [ --help ]                         Produce this help message.
  -v [ --version ]                      Print the version number.
  --binary-patching arg (=yes)          Control the binary patching of the device code.
                                        Disabling this option will result in a loss of precision for error reporting.
  --check-api-memory-access arg (=yes)  Check cudaMemcpy/cudaMemset for accesses to device memory
  --check-device-heap arg (=yes)        Check allocation on the device heap.
  --check-exit-code arg (=yes)          Check application exit code and print an error if it is different than 0
  --check-optix-leaks arg (=no)         Makes the compute-sanitizer track OptiX resources in order to find leaking resources at OptixDeviceContextDestroy time.
  --demangle arg (=full)         

In [ ]:
!compute-sanitizer ./cuda_profile

========= COMPUTE-SANITIZER
cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
========= ERROR SUMMARY: 0 errors


In [ ]:
!./cuda_profile

cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000


In [ ]:
!nvprof --print-gpu-trace ./cuda_profile

==2208== NVPROF is profiling process 2208, command: ./cuda_profile
cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
==2208== Profiling application: ./cuda_profile
==2208== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream        Unified Memory  Virtual Address  Name
302.32ms  35.712us                    -               -         -         -         -  390.63KB  10.431GB/s    Pageable      Device     Tesla T4 (0)         1         7                     -                -  [CUDA memcpy HtoD]
302.44ms  35.231us                    -               -         -         -         -  390.63KB  10.574GB/s    Pageable      Device     Tesla T4 (0)         1         7                     -                -  [CUDA memcpy HtoD]
302.49ms  8.8000us             (98 1 1)      (1024 1 1)        16        0B 

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb


--2023-10-23 14:24:16--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M   231MB/s    in 1.3s    

2023-10-23 14:24:18 (231 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]



In [ ]:
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install


In [ ]:
!nsys profile cuda_profile

cpu control: 		18108465020928.000000
gpu control: 		18108465020928.000000
gpu um control: 	18108465020928.000000
Generating '/tmp/nsys-report-3696.qdstrm'
[1/1] [========================100%] report1.nsys-rep
Generated:
    /content/report1.nsys-rep


In [ ]:
!ncu --set full --call-stack --nvtx -o cuda_profile_stack ./cuda_profile

==PROF== Connected to process 13916 (/content/cuda_profile)
cpu control: 		18108465020928.000000
==PROF== Profiling "gpu_testfunction" - 0: 0%....50%....100% - 31 passes
gpu control: 		18108465020928.000000
==PROF== Profiling "gpu_testfunction_uni_mem" - 1: 0%....50%....100% - 31 passes
gpu um control: 	18108465020928.000000
==PROF== Disconnected from process 13916
==PROF== Report: /content/cuda_profile1.ncu-rep


In [ ]:
!wget "https://bj.bcebos.com/v1/ai-studio-online/9805dd2d2e8e472693efac637628e16b9f9c5be0fe30438bb4a80de3b386781a?responseContentDisposition=attachment%3B%20filename%3DSTI2_1017.zip&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2023-10-18T12%3A42%3A27Z%2F-1%2F%2F6b5388dcd9013bc9b340bb1806476afa938ce0c65f2f595e1a75f529e90e4187" -O STI2_1017.zip

In [ ]:
!rm -rf STI2 && unzip STI2_1017.zip && mv STI2\ 2 STI2

In [ ]:
!ls -gh STI2/translate/docs.txt

-rw-r--r-- 1 root 3.1G Oct  7 07:33 STI2/translate/docs.txt


In [ ]:
!wc -l  STI2/translate/docs.txt
!cat STI2/translate/docs.txt |head
!cat STI2/translate/docs.txt |tail

7853052 STI2/translate/docs.txt

1812
4
42
49
2224
41031
13
1408
40206
21, 22, 23, 25, 26, 27, 29, 1711, 1757, 1841, 1910, 1967, 1971, 1991, 2005, 2051, 2059, 2079, 2117, 2128, 2141, 2188, 2306, 2352, 2453, 2470, 2516, 2518, 2562, 2564, 2573, 2631, 2649, 2650, 2656, 2658, 2682, 2721, 2725, 2730, 2751, 2753, 2756, 2835, 2870, 2875, 2886, 2928, 2954, 3010, 3034, 3035, 3037, 3054, 3110, 3114, 3138, 3156, 3167, 3221, 3248, 3250, 3254, 3273, 3274, 3284, 3294, 3318, 3330, 3342, 3363, 3374, 3427, 3452, 3468, 3472, 3491, 3503, 3530, 3535, 3553, 3588, 3596, 3609, 3667, 3682, 3689, 3719, 3722, 3735, 3752, 3771, 3781, 3793, 3831, 3859, 3883, 3887, 3932, 4021, 4055, 4084, 4098, 4238, 4262, 4295, 4299, 4373, 4499, 4626, 4650, 4791, 4796, 4813, 4848, 5060, 5088, 5174, 5278, 5463, 5776, 22466, 22471, 22482, 30054, 33641, 39222
21, 22, 23, 25, 26, 27, 29, 1660, 1668, 1711, 1834, 1841, 1967, 1971, 1991, 2005, 2051, 2059, 2079, 2117, 2188, 2286, 2306, 2352, 2453, 2516, 2518, 2520, 2562, 2564, 2573, 2649

In [ ]:
!cat STI2/translate/res/result.txt

2990
2095355	2104387	2118767	2147776	2055923	2206974	2238603	2020057	2267746	2220291	2151279	2154228	1960823	2001825	2005314	2029722	2054064	2098358	2099815	2287637	2300031	2158918	2167292	2177721	2177855	1981772	2196213	2520566	1917947	2017068	2028496	2083981	2323778	2247926	2282000	2202772	2227479	2236897	2186351	2486485	2150430	2381725	1760981	1855263	1869531	1877846	1885219	1896166	1950185	2028110	2079765	2119578	2126999	2306247	2315934	1776753	1811788	1848411	1901590	2026760	2042826	2073036	2217999	2224835	1709996	1754639	1844734	1987029	2091827	2093706	2284846	2151301	2328000	2351661	1706744	2081563	2597490	1759061	2011063	2077727	2737847	2362721	1956654	2193430	2236145	1462218	1816978	1897084	1897296	1902180	1925148	1933929	1952948	1958058	1975060	1984389	1984524	1987018	2013793	2034453
3354548	3584604	3385811	3457483	3229654	3732526	3320427	3340108	2491259	3743142	3631235	3648390	3657624	3516117	2374643	2403142	2575906	2228660	4836663	1839330	4591327	3557359	4697352	3405025	357

In [ ]:
!sh STI2/build.sh

build success


In [ ]:
!rm nohup.out
!nohup STI2/bin/query_doc_scoring ./STI2/translate/docs.txt ./STI2/translate/querys ./res.txt &


rm: cannot remove 'nohup.out': No such file or directory
nohup: appending output to 'nohup.out'


In [ ]:
!cat nohup.out

start get topk
query1.txt:10, 11, 16, 17, 42, 60, 22524, 22546, 22590, 22784, 23212, 23427, 23485, 23525, 23554, 24129, 24133, 24645, 24804, 24875, 25129, 25242, 25502, 25705, 25994, 26000, 26045, 26046, 26077, 26114, 26247, 26338, 26407, 27263, 27468, 27513, 28100, 40111, 40228, 40388, 41700, 45156, 45946, 46367, 47181, 47460, 47672
query2.txt:10, 16, 18, 21, 22, 23, 30, 42, 43, 44, 45, 54, 22497, 22512, 22524, 22533, 22535, 22608, 22624, 22790, 22828, 22836, 22885, 23188, 23381, 23409, 23558, 24103, 24197, 24250, 24496, 24918, 24974, 24987, 25179, 25317, 25827, 25994, 25996, 26009, 26015, 26023, 26030, 26050, 26052, 26082, 26096, 26205, 26247, 27399, 27475, 40029, 40300, 40416, 40504, 40696, 40837, 41166, 41172, 41336, 41407, 41516, 43247, 43309, 44547, 44795, 45101, 48828
query3.txt:11, 12, 13, 14, 21, 22, 23, 33, 42, 53, 61, 1380, 1545, 1546, 1557, 1560, 1566, 1569, 1583, 1646, 1759, 1762, 1787, 1794, 1877, 1882, 1892, 2069, 2120, 2146, 2368, 2670, 2888, 3022, 3327, 3335, 22460, 22

In [ ]:
!cat res.txt

3147
2095355	2104387	2118767	2147776	2055923	2206974	2238603	2020057	2267746	2220291	2151279	2154228	1960823	2001825	2005314	2029722	2054064	2098358	2099815	2287637	2300031	2158918	2167292	2177721	2177855	1981772	2196213	2520566	1917947	2017068	2028496	2083981	2323778	2247926	2282000	2202772	2227479	2236897	2186351	2486485	2150430	2381725	1760981	1855263	1869531	1877846	1885219	1896166	1950185	2028110	2079765	2119578	2126999	2306247	2315934	1776753	1811788	1848411	1901590	2026760	2042826	2073036	2217999	2224835	1709996	1754639	1844734	1987029	2091827	2093706	2284846	2151301	2328000	2351661	1706744	2081563	2597490	1759061	2011063	2077727	2737847	2362721	1956654	2193430	2236145	1462218	1816978	1897084	1897296	1902180	1925148	1933929	1952948	1958058	1975060	1984389	1984524	1987018	2013793	2034453
3354548	3584604	3385811	3457483	3229654	3732526	3320427	3340108	2491259	3743142	3631235	3648390	3657624	3516117	2374643	2403142	2575906	2228660	4836663	1839330	4591327	3557359	4697352	3405025	357

In [ ]:
!diff res.txt STI2/translate/res/result.txt

1c1
< 3147
---
> 2990


In [ ]:
!nvcc --help


Usage  : nvcc [options] <inputfile>

Options for specifying the compilation phase
More exactly, this option specifies up to which stage the input files must be compiled,
according to the following compilation trajectories for different input file types:
        .c/.cc/.cpp/.cxx : preprocess, compile, link
        .o               : link
        .i/.ii           : compile, link
        .cu              : preprocess, cuda frontend, PTX assemble,
                           merge with host C code, compile, link
        .gpu             : cicc compile into cubin
        .ptx             : PTX assemble into cubin.

--cuda                                          (-cuda)                         
        Compile all .cu input files to .cu.cpp.ii output.

--cubin                                         (-cubin)                        
        Compile all .cu/.gpu/.ptx input files to device-only .cubin files.  This
        step discards the host code for each .cu input file.

--fatbin          

In [ ]:
!compute-sanitizer ./STI2/bin/query_doc_scoring ./STI2/translate/docs.txt ./STI2/translate/querys ./res.txt

In [43]:
!nvprof --help

Usage: nvprof [options] [application] [application-arguments]
Options:
       --aggregate-mode <on|off>
                        Turn on/off aggregate mode for events and metrics specified
                        by subsequent "--events" and "--metrics" options. Those
                        event/metric values will be collected for each domain instance,
                        instead of the whole device. Allowed values:
                        	on - turn on aggregate mode (default)
                        	off - turn off aggregate mode

       --analysis-metrics
                        Collect profiling data that can be imported to Visual Profiler's
                        "analysis" mode. Note: Use "--export-profile" to specify
                        an export file.

       --annotate-mpi <off|openmpi|mpich>
                        Automatically annotate MPI calls with NVTX markers. Specify
                        the MPI implementation installed on your machine. Currently,
        

In [44]:
!nvprof --print-gpu-trace ./STI2/bin/query_doc_scoring ./STI2/translate/docs.txt ./STI2/translate/querys ./res.txt

start get topk
query1.txt:10, 11, 16, 17, 42, 60, 22524, 22546, 22590, 22784, 23212, 23427, 23485, 23525, 23554, 24129, 24133, 24645, 24804, 24875, 25129, 25242, 25502, 25705, 25994, 26000, 26045, 26046, 26077, 26114, 26247, 26338, 26407, 27263, 27468, 27513, 28100, 40111, 40228, 40388, 41700, 45156, 45946, 46367, 47181, 47460, 47672
query2.txt:10, 16, 18, 21, 22, 23, 30, 42, 43, 44, 45, 54, 22497, 22512, 22524, 22533, 22535, 22608, 22624, 22790, 22828, 22836, 22885, 23188, 23381, 23409, 23558, 24103, 24197, 24250, 24496, 24918, 24974, 24987, 25179, 25317, 25827, 25994, 25996, 26009, 26015, 26023, 26030, 26050, 26052, 26082, 26096, 26205, 26247, 27399, 27475, 40029, 40300, 40416, 40504, 40696, 40837, 41166, 41172, 41336, 41407, 41516, 43247, 43309, 44547, 44795, 45101, 48828
query3.txt:11, 12, 13, 14, 21, 22, 23, 33, 42, 53, 61, 1380, 1545, 1546, 1557, 1560, 1566, 1569, 1583, 1646, 1759, 1762, 1787, 1794, 1877, 1882, 1892, 2069, 2120, 2146, 2368, 2670, 2888, 3022, 3327, 3335, 22460, 22

In [38]:
!nsys --help

 usage: nsys [--version] [--help] <command> [<args>] [application] [<application args>]

 The most commonly used nsys commands are:
	profile       Run an application and capture its profile into a QDSTRM file.
	launch        Launch an application ready to be profiled.
	start         Start a profiling session.
	stop          Stop a profiling session and capture its profile into a QDSTRM file.
	cancel        Cancel a profiling session and discard any collected data.
	service       Launch the Nsight Systems data service.
	stats         Generate statistics from an existing nsys-rep or SQLite file.
	status        Provide current status of CLI or the collection environment.
	shutdown      Disconnect launched processes from the profiler and shutdown the profiler.
	sessions list List active sessions.
	export        Export nsys-rep file into another format.
	analyze       Identify optimization opportunities in a nsys-rep or SQLITE file.
	recipe        Run a recipe for multi-node analysis.
	nvpr

In [42]:
!nsys --help profile


usage: nsys profile [<args>] [application] [<application args>]

	-b, --backtrace=

	   Possible values are 'lbr', 'fp', 'dwarf', or 'none'.
	   Select the backtrace method to use while sampling.
	   Select 'none' to disable backtrace collection.
	   Default is 'lbr'.

	-c, --capture-range=

	   Possible values are none, cudaProfilerApi, nvtx, hotkey.
	   When '-c cudaProfilerApi' is used, profiling will start only when cudaProfilerStart API is
	   invoked in the application.
	   When '-c nvtx' is used, profiling will start only when the specified NVTX range is
	   started in the application.
	   When '-c hotkey' is used, profiling will start only when the hotkey
	   set by '--hotkey-capture' is pressed in the application. This works for graphic apps only.
	   Note that you must enable CUDA or NVTX tracing of the target application
	   for '-c cudaProfilerApi' or '-c nvtx' to work.
	   When '-capture-range none' is used, cudaProfilerStart/Stop APIs and hotkey will 
	   be ignored and 

In [39]:
!nsys profile ./STI2/bin/query_doc_scoring ./STI2/translate/docs.txt ./STI2/translate/querys ./res.txt

start get topk
query1.txt:10, 11, 16, 17, 42, 60, 22524, 22546, 22590, 22784, 23212, 23427, 23485, 23525, 23554, 24129, 24133, 24645, 24804, 24875, 25129, 25242, 25502, 25705, 25994, 26000, 26045, 26046, 26077, 26114, 26247, 26338, 26407, 27263, 27468, 27513, 28100, 40111, 40228, 40388, 41700, 45156, 45946, 46367, 47181, 47460, 47672
query2.txt:10, 16, 18, 21, 22, 23, 30, 42, 43, 44, 45, 54, 22497, 22512, 22524, 22533, 22535, 22608, 22624, 22790, 22828, 22836, 22885, 23188, 23381, 23409, 23558, 24103, 24197, 24250, 24496, 24918, 24974, 24987, 25179, 25317, 25827, 25994, 25996, 26009, 26015, 26023, 26030, 26050, 26052, 26082, 26096, 26205, 26247, 27399, 27475, 40029, 40300, 40416, 40504, 40696, 40837, 41166, 41172, 41336, 41407, 41516, 43247, 43309, 44547, 44795, 45101, 48828
query3.txt:11, 12, 13, 14, 21, 22, 23, 33, 42, 53, 61, 1380, 1545, 1546, 1557, 1560, 1566, 1569, 1583, 1646, 1759, 1762, 1787, 1794, 1877, 1882, 1892, 2069, 2120, 2146, 2368, 2670, 2888, 3022, 3327, 3335, 22460, 22

In [40]:
!ncu --help

usage: ncu [options] [program] [program-arguments]

General Options:
  -h [ --help ]                         Print this help message.
  -v [ --version ]                      Print the version number.
  --mode arg (=launch-and-attach)       Select the mode of interaction with the target application:
                                          launch-and-attach
                                          (launch and attach for profiling)
                                          launch
                                          (launch and suspend for later attach)
                                          attach
                                          (attach to launched application)
  -p [ --port ] arg (=49152)            Base port for connecting to target application
  --max-connections arg (=64)           Maximum number of ports for connecting to target application

Launch Options:
  --check-exit-code arg (=1)            Check the application exit code and print an error if it is differ

In [41]:
!ncu --set full --call-stack --nvtx -o cuda_topk_profile_stack ./STI2/bin/query_doc_scoring ./STI2/translate/docs.txt ./STI2/translate/querys ./res.txt

start get topk
query1.txt:10, 11, 16, 17, 42, 60, 22524, 22546, 22590, 22784, 23212, 23427, 23485, 23525, 23554, 24129, 24133, 24645, 24804, 24875, 25129, 25242, 25502, 25705, 25994, 26000, 26045, 26046, 26077, 26114, 26247, 26338, 26407, 27263, 27468, 27513, 28100, 40111, 40228, 40388, 41700, 45156, 45946, 46367, 47181, 47460, 47672
query2.txt:10, 16, 18, 21, 22, 23, 30, 42, 43, 44, 45, 54, 22497, 22512, 22524, 22533, 22535, 22608, 22624, 22790, 22828, 22836, 22885, 23188, 23381, 23409, 23558, 24103, 24197, 24250, 24496, 24918, 24974, 24987, 25179, 25317, 25827, 25994, 25996, 26009, 26015, 26023, 26030, 26050, 26052, 26082, 26096, 26205, 26247, 27399, 27475, 40029, 40300, 40416, 40504, 40696, 40837, 41166, 41172, 41336, 41407, 41516, 43247, 43309, 44547, 44795, 45101, 48828
query3.txt:11, 12, 13, 14, 21, 22, 23, 33, 42, 53, 61, 1380, 1545, 1546, 1557, 1560, 1566, 1569, 1583, 1646, 1759, 1762, 1787, 1794, 1877, 1882, 1892, 2069, 2120, 2146, 2368, 2670, 2888, 3022, 3327, 3335, 22460, 22